In [1]:
!pip install spacy


In [13]:
!pip install newsapi-python

In [5]:
!python -m spacy download en_core_web_lg

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [3]:
import spacy
import en_core_web_lg
from newsapi.newsapi_client import NewsApiClient

In [4]:
nlp_eng = en_core_web_lg.load()
newsapi = NewsApiClient (api_key='db53c380ff8c41c487b4f973f97deadc')

In [24]:
articles = []
for x in range(5):
    articles.append(newsapi.get_everything(q='coronavirus', language='en', 
                                  from_param='2020-09-29', to='2020-10-27', sort_by='relevancy',page=x+1))

In [27]:
print(articles[4])

{'status': 'ok', 'totalResults': 172364, 'articles': [{'source': {'id': None, 'name': 'New York Times'}, 'author': 'Bryan Pietsch', 'title': 'Coronavirus Tracers Beg Residents: Please Answer Your Phones', 'description': 'Spam calls and scammers are more than annoying; they’re hindering efforts to track the coronavirus.', 'url': 'https://www.nytimes.com/2020/09/29/us/covid-contact-tracing.html', 'urlToImage': 'https://static01.nyt.com/images/2020/09/28/multimedia/28xp-contacttracing/merlin_174849945_f345bcdd-818d-4e73-9a4e-fe244a7760ae-facebookJumbo.jpg', 'publishedAt': '2020-09-29T14:34:12Z', 'content': 'Thats far better than what many municipalities are reporting, Dr. Cannuscio said.\r\nTracers in Louisiana were able to reach 66 percent of identified cases from May 15 to Sept. 24, Alyson Neel, a spoke… [+1443 chars]'}, {'source': {'id': None, 'name': 'New York Times'}, 'author': 'Farhad Manjoo', 'title': 'Why Won’t Trump Tell Americans to Wear Masks?', 'description': 'How a simple, ef

In [36]:
data = []

for article in articles:
    for x in article['articles']:
        title = x['title']
        desc = x['description']
        content = x['content']
        data.append({'title': title, 'description': desc, 'content':content})
    
print(len(data))

100


In [37]:
import pandas as pd

df = pd.DataFrame(data)
df = df.dropna()
df.head()

,title,description,content
0,"Sorry, Mouthwash Isn't a Coronavirus Cure",Mouthwash is designed to kill germs in your mo...,Mouthwash is designed to kill germs in your mo...
1,Check an Airline's Coronavirus Policies Before...,Navigating air travel during the COVID-19 pand...,Navigating air travel during the COVID-19 pand...
2,"Twitter flags, limits sharing on Trump tweet a...",Twitter disabled sharing options on a tweet fr...,The tweet violated the platforms rules against...
3,Coronavirus and Home Design,A recent survey of architects reveals that cli...,The coronavirus pandemic is already shaping ho...
4,Coronavirus Briefing: What Happened Today,What the near future may hold,Many leaders most notably President Trump have...


In [39]:
from collections import Counter

results = []

def get_keywords_eng(data):
    words = []
    doc = nlp_eng(data)
    punctuation = [",",".","!","?"]
    pos_tag = ["VERB", "NOUN", "PROPN"]
    for token in doc:
        if (token.text in nlp_eng.Defaults.stop_words or token.text in punctuation):
            continue
        if (token.pos_ in pos_tag):
            words.append(token.text)
    return words
    
for content in df.content.values:
    results.append([('#' + x[0]) for x in Counter(get_keywords_eng(content)).most_common(5)])
    
df['keywords'] = results

df.head()

,title,description,content,keywords
0,"Sorry, Mouthwash Isn't a Coronavirus Cure",Mouthwash is designed to kill germs in your mo...,Mouthwash is designed to kill germs in your mo...,"[#Mouthwash, #designed, #kill, #germs, #mouth]"
1,Check an Airline's Coronavirus Policies Before...,Navigating air travel during the COVID-19 pand...,Navigating air travel during the COVID-19 pand...,"[#Navigating, #air, #travel, #COVID-19, #pande..."
2,"Twitter flags, limits sharing on Trump tweet a...",Twitter disabled sharing options on a tweet fr...,The tweet violated the platforms rules against...,"[#tweet, #sharing, #violated, #platforms, #rules]"
3,Coronavirus and Home Design,A recent survey of architects reveals that cli...,The coronavirus pandemic is already shaping ho...,"[#pandemic, #coronavirus, #shaping, #home, #de..."
4,Coronavirus Briefing: What Happened Today,What the near future may hold,Many leaders most notably President Trump have...,"[#leaders, #President, #Trump, #described, #ar..."
